In [4]:
import numpy as np

data = np.load("/home/elena/emcomm/objects_game_test_dataset/[4, 4, 4, 4, 4]_3_distractors.npz")

In [5]:
data

NpzFile '/home/elena/emcomm/objects_game_test_dataset/[4, 4, 4, 4, 4]_3_distractors.npz' with keys: train, train_labels, valid, valid_labels, test...

In [6]:
print(data.files)

['train', 'train_labels', 'valid', 'valid_labels', 'test', 'test_labels', 'n_distractors']


In [7]:
for k in data.files:
    print(k, data[k].shape)


train (2000, 4, 5)
train_labels (2000,)
valid (100, 4, 5)
valid_labels (100,)
test (100, 4, 5)
test_labels (100,)
n_distractors ()


In [13]:
print("First train tuple:\n", data["train"][2])
print("First train label:", data["train_labels"][2])


First train tuple:
 [[4 4 4 3 3]
 [4 4 4 2 3]
 [2 2 2 3 3]
 [1 1 2 4 4]]
First train label: 0


In [ ]:
import pprint
pprint.pprint(data["train"][0])

array([[4, 2, 1, 3, 2],
       [4, 2, 4, 1, 3],
       [1, 2, 1, 2, 2],
       [2, 1, 3, 3, 3]])


In [17]:
from datasets import load_from_disk
ds = load_from_disk("/home/elena/emcomm/datasets/coco_val_features")
ds

Dataset({
    features: ['captions', 'features'],
    num_rows: 5000
})

In [18]:
import numpy as np

features = np.array(ds["features"])

In [19]:
def create_tuples(features, n_samples, n_distractors=3):
    """
    features: np.array of shape (num_objects, n_features)
    n_samples: how many tuples to generate
    n_distractors: number of distractor objects per tuple
    """
    num_objects = features.shape[0]
    tuple_size = n_distractors + 1

    tuples = []
    labels = []

    rng = np.random.default_rng()  # random generator

    for _ in range(n_samples):
        # Pick tuple_size unique objects
        idxs = rng.choice(num_objects, size=tuple_size, replace=False)
        tuple_vectors = features[idxs]

        # Pick a random index in the tuple as the target
        target_idx = rng.integers(0, tuple_size)

        tuples.append(tuple_vectors)
        labels.append(target_idx)

    return np.array(tuples), np.array(labels)


In [20]:
train_tuples, train_labels = create_tuples(np.array(ds["features"]), n_samples=20, n_distractors=3)
valid_tuples, valid_labels = create_tuples(np.array(ds["features"]), n_samples=10, n_distractors=3)
test_tuples, test_labels   = create_tuples(np.array(ds["features"]), n_samples=10, n_distractors=3)


In [24]:
train_tuples.shape

(20, 4, 1000)

In [25]:
np.savez_compressed(
    "../../../datasets/coco_val.npz",
    train=train_tuples,
    train_labels=train_labels,
    valid=valid_tuples,
    valid_labels=valid_labels,
    test=test_tuples,
    test_labels=test_labels,
    n_distractors=3
)